In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#%cd drive/MyDrive/xmlc_research/
#!git clone https://github.com/ThejaniYapa/attention_xml.git
%cd /content/drive/MyDrive/xmlc_research/attention_xml
!ls
#%env GIT_DISCOVERY_ACROSS_FILESYSTEM=1
#!git checkout -b dev

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

#Downloading data

In [ ]:
#https://drive.google.com/open?id=10w_HuLklGc8GA_FtUSdnHT8Yo1mxYziP&authuser=0
%cd data
#!gdown --id 10w_HuLklGc8GA_FtUSdnHT8Yo1mxYziP
!unzip glove.zip
%cd ..

In [ ]:
!head -50 /content/drive/MyDrive/xmlc_research/attention_xml/data/Amazon-670K/test_labels.txt

In [ ]:
# Download dataset from : https://drive.google.com/file/d/1Xd4BPFy1RPmE7MEXMu77E2_xWOhR1pHW/view
%cd data
#!gdown --id 1Xd4BPFy1RPmE7MEXMu77E2_xWOhR1pHW
!tar -xvzf Amazon-670K.tar.gz
#!unzip Datasets/Amazon-670K.zip
%cd ..

In [ ]:
!pwd

In [ ]:
def read_and_save_first_n_lines(file_path, n, output_file_path):
    lines_to_save = []

    with open(file_path, "r") as file:
        for _ in range(n):
            line = file.readline()
            if not line:
                break  # In case the file has fewer than n lines
            lines_to_save.append(line)

    with open(output_file_path, "w") as output_file:
        for line in lines_to_save:
            output_file.write(line)


read_and_save_first_n_lines("data/Amazon-670K/test_labels.txt",500,"data/Amazon-670K/test/test_labels.txt")#input_file_path, 5, output_file_path
read_and_save_first_n_lines("data/Amazon-670K/test_raw_texts.txt",500,"data/Amazon-670K/test/test_raw_texts.txt")
read_and_save_first_n_lines("data/Amazon-670K/train_labels.txt",500,"data/Amazon-670K/test/train_labels.txt")
read_and_save_first_n_lines("data/Amazon-670K/train_raw_texts.txt",500,"data/Amazon-670K/test/train_raw_texts.txt")

In [ ]:
!cp data/Amazon-670K/train_v1.txt data/Amazon-670K/test/train_v1.txt

#Preprocessing

## went wrong

In [ ]:
# Download the GloVe embedding (840B,300d) and convert it to gensim format (which can be loaded by gensim.models.KeyedVectors.load).
#!wget https://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip 'glove.840B.300d.zip'


In [ ]:
#https://github.com/jroakes/glove-to-word2vec/blob/master/convert.py
!python gensim_convert.py

In [ ]:
#https://www.techunits.com/topics/neural-networks/shallow-neural-network/how-to-convert-a-pre-trained-glove-vector-into-word2vec-format/
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/content/drive/MyDrive/xmlc_research/attention_xml/glove.840B.300d.txt')
# create temp file and save converted embedding into it
target_file = get_tmpfile('/content/drive/MyDrive/xmlc_research/attention_xml/word2vec.840B.300d.txt')
#glove2word2vec('glove.840B.300d.txt', target_file)

glove2word2vec(glove_file, target_file)
model = KeyedVectors.load_word2vec_format(target_file)


Both the GloVe algorithm and word2vec both create word-vectors, a vector per word.

But the formats for storing those vectors are slightly different. The gensim glove2word2vec() function will let you convert a file in GloVe format to the format used by the original Google word2vec.c code.

https://radimrehurek.com/gensim/scripts/glove2word2vec.html

Meanwhile, the gensim KeyedVectors.load_word2vec_format() method can load vectors in that word2vec.c format, into an instance of KeyedVectors (or one of its same-interface subclasses), for easy lookup and other common word-vector operations.

In [ ]:
# Demo: Loads the newly created glove_model.txt into gensim API.
from gensim.models import KeyedVectors
#model=KeyedVectors.load_word2vec_format('glove_model.gensim.txt',binary=False) #GloVe Model
#model = KeyedVectors.load('glove.840B.300d.txt')
#print (glove_w2vec.most_similar(positive=['australia'], topn=10))
print (model.similarity('woman', 'man'))

In [ ]:
!head -10 /content/drive/MyDrive/xmlc_research/attention_xml/glove_model.gensim.txt
!head -10 /content/drive/MyDrive/xmlc_research/attention_xml/glove.840B.300d.txt

In [ ]:
import nltk
nltk.download('punkt')

## Testing

In [ ]:
!python preprocess.py \
--text-path data/Amazon-670K/test/train_raw_texts.txt \
--tokenized-path data/Amazon-670K/test/train_texts.txt \
--label-path data/Amazon-670K/test/train_labels.txt \
--vocab-path data/Amazon-670K/test/vocab.npy \
--emb-path data/Amazon-670K/test/emb_init.npy \
--w2v-model data/glove.840B.300d.gensim

In [ ]:
!python preprocess.py \
--text-path data/Amazon-670K/test/test_raw_texts.txt \
--tokenized-path data/Amazon-670K/test/test_texts.txt \
--label-path data/Amazon-670K/test/test_labels.txt \
--vocab-path data/Amazon-670K/test/vocab.npy

# Train and Predict

In [ ]:
#python main.py --data-cnf configure/datasets/EUR-Lex.yaml --model-cnf configure/models/AttentionXML-EUR-Lex.yaml
!python main.py --data-cnf configure/datasets/Amazon-670K.yaml --model-cnf configure/models/AttentionXML-Amazon-670K.yaml

In [ ]:
!git status

In [ ]:
!git branch
!git branch -a

In [ ]:
!git add configure/models/AttentionXML-Amazon-670K.yaml

In [ ]:
!git restore --staged configure/models/AttentionXML-Amazon-670K.yaml

In [ ]:
!cp /content/drive/MyDrive/xmlc_research/attention_main.ipynb /content/drive/MyDrive/xmlc_research/attention_xml/requirements.txt